<a href="https://colab.research.google.com/github/amirhossini/Pyomo-Educational-Notebooks/blob/main/Pyomo2_NLP.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Pyomo Examples

__Notebook:__ Non-Linear Prpogramming (NLP) 

__Questions:__ amir.hossini@queensu.ca

_Libraries_

In [1]:
! pip install pyomo                 # software package for formulating optimization problems
! apt-get install -y -qq glpk-utils # software package with solver for large scale LP and MILP problems
import pyomo.environ as pyomo

! wget -N -q "https://ampl.com/dl/open/ipopt/ipopt-linux64.zip"
! unzip -o -q ipopt-linux64

     |████████████████████████████████| 9.6 MB 14.0 MB/s 
     |████████████████████████████████| 49 kB 5.2 MB/s 
Selecting previously unselected package libsuitesparseconfig5:amd64.
(Reading database ... 155455 files and directories currently installed.)
Preparing to unpack .../libsuitesparseconfig5_1%3a5.1.2-2_amd64.deb ...
Unpacking libsuitesparseconfig5:amd64 (1:5.1.2-2) ...
Selecting previously unselected package libamd2:amd64.
Preparing to unpack .../libamd2_1%3a5.1.2-2_amd64.deb ...
Unpacking libamd2:amd64 (1:5.1.2-2) ...
Selecting previously unselected package libcolamd2:amd64.
Preparing to unpack .../libcolamd2_1%3a5.1.2-2_amd64.deb ...
Unpacking libcolamd2:amd64 (1:5.1.2-2) ...
Selecting previously unselected package libglpk40:amd64.
Preparing to unpack .../libglpk40_4.65-1_amd64.deb ...
Unpacking libglpk40:amd64 (4.65-1) ...
Selecting previously unselected package glpk-utils.
Preparing to unpack .../glpk-utils_4.65-1_amd64.deb ...
Unpacking glpk-utils (4.65-1) ...
Setting up

### Example 1 - Regreassion

In [2]:
#### Model definition
model_linear = pyomo.ConcreteModel()
model_nonlinear = pyomo.ConcreteModel()

#### Set declaration
model_linear.m = pyomo.RangeSet(6)
model_nonlinear.m = pyomo.RangeSet(6)

#### Variable definition
model_linear.a1 = pyomo.Var(domain=pyomo.Reals)
model_linear.a2 = pyomo.Var(domain=pyomo.Reals)

model_nonlinear.b1 = pyomo.Var(domain=pyomo.Reals, initialize = 500)
model_nonlinear.b2 = pyomo.Var(domain=pyomo.Reals, initialize = -150)
model_nonlinear.b3 = pyomo.Var(domain=pyomo.Reals, initialize = -0.2, bounds=(-5,5))

#### Parameter declaration 
model_linear.datapoints_y = pyomo.Param(model_linear.m, 
                                        initialize = {1:127,2:151,3:379,
                                                      4:421,5:460,6:426})
model_linear.datapoints_x = pyomo.Param(model_linear.m, 
                                        initialize = {1:-5,2:-3,3:-1,
                                                      4:5,5:3,6:1})

model_nonlinear.datapoints_y = pyomo.Param(model_nonlinear.m, 
                                        initialize = {1:127,2:151,3:379,
                                                      4:421,5:460,6:426})
model_nonlinear.datapoints_x = pyomo.Param(model_nonlinear.m, 
                                        initialize = {1:-5,2:-3,3:-1,
                                                      4:5,5:3,6:1})

#### Objective functions
model_linear.obj = pyomo.Objective(expr = sum((model_linear.datapoints_y[m]-
                                               (model_linear.a1+model_linear.a2*model_linear.datapoints_x[m]))**2 
                                              for m in model_linear.m),sense = pyomo.minimize)

model_nonlinear.obj = pyomo.Objective(expr = sum((model_nonlinear.datapoints_y[m]-
                                               (model_nonlinear.b1+model_nonlinear.b2*pyomo.exp(model_nonlinear.b3*model_nonlinear.datapoints_x[m])))**2 
                                              for m in model_nonlinear.m),sense = pyomo.minimize)

#### Solver options
results = pyomo.SolverFactory('ipopt', executable = '/content/ipopt').solve(model_linear)

results.write()
print("\n RESULTS \n")
print("Squared deviation for linear regression model = ", model_linear.obj())
print('Coefficient 1 for linear regression (a1) = ', model_linear.a1())
print('Coefficient 2 for linear regression (a2) = ', model_linear.a2())

results = pyomo.SolverFactory('ipopt', executable = '/content/ipopt').solve(model_nonlinear)

results.write()
print("\n RESULTS \n")
print("Squared deviation for nonlinear regression model = ", model_nonlinear.obj())
print('Coefficient 1 for linear regression (b1) = ', model_nonlinear.b1())
print('Coefficient 2 for linear regression (b2) = ', model_nonlinear.b2())
print('Coefficient 2 for linear regression (b3) = ', model_nonlinear.b3())

# ==========================================================
# = Solver Results                                         =
# ==========================================================
# ----------------------------------------------------------
#   Problem Information
# ----------------------------------------------------------
Problem: 
- Lower bound: -inf
  Upper bound: inf
  Number of objectives: 1
  Number of constraints: 0
  Number of variables: 2
  Sense: unknown
# ----------------------------------------------------------
#   Solver Information
# ----------------------------------------------------------
Solver: 
- Status: ok
  Message: Ipopt 3.12.13\x3a Optimal Solution Found
  Termination condition: optimal
  Id: 0
  Error rc: 0
  Time: 0.018710851669311523
# ----------------------------------------------------------
#   Solution Information
# ----------------------------------------------------------
Solution: 
- number of solutions: 0
  number of solutions displayed: 0

 RESU

### Example 2 - Geometirc Optimization

In [3]:
#### Model definition
model_circle = pyomo.ConcreteModel()

#### Set defintion
model_circle.i = pyomo.RangeSet(5)
model_circle.j = pyomo.Set(initialize = model_circle.i) # Creates an alias of set i

#### Variable declaration
model_circle.r = pyomo.Var(domain = pyomo.NonNegativeReals, bounds = (0.05,0.4))

#### Function to initialize coordinate variables
def init(model_circle,l):
  return -0.2 + l * 0.1

model_circle.x = pyomo.Var(model_circle.i,domain=pyomo.Reals,bounds=(-1,1),initialize = init)
model_circle.y = pyomo.Var(model_circle.i,domain=pyomo.Reals,bounds=(-1,1),initialize = init)

#### Constrain definition
def rule4(model_circle,i):
  return (1-model_circle.r)**2 >= (model_circle.x[i]**2)+(model_circle.y[i]**2)
model_circle.eq4 = pyomo.Constraint(model_circle.i,rule=rule4,doc='Containment constraint')

def rule5(model_circle,i,j):
  if i<j:
    return (model_circle.x[i]-model_circle.x[j])**2 + (model_circle.y[i]-model_circle.y[j])**2 >= 4*(model_circle.r**2)
  return pyomo.Constraint.Skip

model_circle.eq5 = pyomo.Constraint(model_circle.i,model_circle.j, rule = rule5, doc = 'No overlap constraint')

#### Objective function
model_circle.obj = pyomo.Objective(expr = model_circle.r, sense = pyomo.maximize)

#### Solve statement
results = pyomo.SolverFactory('ipopt', executable = '/content/ipopt').solve(model_circle)

#### Printing results
results.write()
print('\n RESULTS \n')
print('Radius of identically size circles = ', model_circle.r(),'\n')
for i in model_circle.i:
  print("Coordinate of circle ",i, "=> (", model_circle.x[i](),",",model_circle.y[i](),") \n")

# ==========================================================
# = Solver Results                                         =
# ==========================================================
# ----------------------------------------------------------
#   Problem Information
# ----------------------------------------------------------
Problem: 
- Lower bound: -inf
  Upper bound: inf
  Number of objectives: 1
  Number of constraints: 15
  Number of variables: 11
  Sense: unknown
# ----------------------------------------------------------
#   Solver Information
# ----------------------------------------------------------
Solver: 
- Status: ok
  Message: Ipopt 3.12.13\x3a Optimal Solution Found
  Termination condition: optimal
  Id: 0
  Error rc: 0
  Time: 0.07117772102355957
# ----------------------------------------------------------
#   Solution Information
# ----------------------------------------------------------
Solution: 
- number of solutions: 0
  number of solutions displayed: 0

 RES

### Example 3 - Resource Allocation

In [4]:
model = pyomo.ConcreteModel()

#### Set declaration
model.w = pyomo.Set(initialize=['ICBM','MRBM-1','LR-Bomber','F-Bomber','MRBM-2'])
model.t = pyomo.RangeSet(20)

model.t_spec = pyomo.Set(within = model.t, initialize=[1, 6, 10, 14, 15, 16, 20])

#### Variable declaration 
model.x=pyomo.Var(model.w,model.t,domain=pyomo.NonNegativeReals)

#### Parameter declaration
model.phi = pyomo.Param(model.w,model.t,initialize = {('ICBM',1):1.00,('ICBM',2):0.95,('ICBM',3):1.00,('ICBM',4):1.00,('ICBM',5):1.00,
                                                      ('ICBM',6):0.85,('ICBM',7):0.90,('ICBM',8):0.85,('ICBM',9):0.80,('ICBM',10):1.00,
                                                      ('ICBM',11):1.00,('ICBM',12):1.00,('ICBM',13):1.00,('ICBM',14):1.00,('ICBM',15):1.00,
                                                      ('ICBM',16):1.00,('ICBM',17):1.00,('ICBM',18):0.95,('ICBM',19):1.00,('ICBM',20):1.00,

                                                      ('MRBM-1',1):0.84,('MRBM-1',2):0.83,('MRBM-1',3):0.85,('MRBM-1',4):0.84,('MRBM-1',5):0.85,
                                                      ('MRBM-1',6):0.81,('MRBM-1',7):0.81,('MRBM-1',8):0.82,('MRBM-1',9):0.80,('MRBM-1',10):0.86,
                                                      ('MRBM-1',11):1.00,('MRBM-1',12):0.98,('MRBM-1',13):1.00,('MRBM-1',14):0.88,('MRBM-1',15):0.87,
                                                      ('MRBM-1',16):0.88,('MRBM-1',17):0.85,('MRBM-1',18):0.84,('MRBM-1',19):0.85,('MRBM-1',20):0.85, 

                                                      ('LR-Bomber',1):0.96,('LR-Bomber',2):0.95,('LR-Bomber',3):0.96,('LR-Bomber',4):0.96,('LR-Bomber',5):0.96,
                                                      ('LR-Bomber',6):0.90,('LR-Bomber',7):0.92,('LR-Bomber',8):0.91,('LR-Bomber',9):0.92,('LR-Bomber',10):0.95,
                                                      ('LR-Bomber',11):0.99,('LR-Bomber',12):0.98,('LR-Bomber',13):0.99,('LR-Bomber',14):0.98,('LR-Bomber',15):0.97,
                                                      ('LR-Bomber',16):0.98,('LR-Bomber',17):0.95,('LR-Bomber',18):0.92,('LR-Bomber',19):0.93,('LR-Bomber',20):0.92,

                                                      ('F-Bomber',1):1.00,('F-Bomber',2):1.00,('F-Bomber',3):1.00,('F-Bomber',4):1.00,('F-Bomber',5):1.00,
                                                      ('F-Bomber',6):1.00,('F-Bomber',7):1.00,('F-Bomber',8):1.00,('F-Bomber',9):1.00,('F-Bomber',10):0.96,
                                                      ('F-Bomber',11):0.91,('F-Bomber',12):0.92,('F-Bomber',13):0.91,('F-Bomber',14):0.92,('F-Bomber',15):0.98,
                                                      ('F-Bomber',16):0.93,('F-Bomber',17):1.00,('F-Bomber',18):1.00,('F-Bomber',19):1.00,('F-Bomber',20):1.00,

                                                      ('MRBM-2',1):0.92,('MRBM-2',2):0.94,('MRBM-2',3):0.92,('MRBM-2',4):0.95,('MRBM-2',5):0.95,
                                                      ('MRBM-2',6):0.98,('MRBM-2',7):0.98,('MRBM-2',8):1.00,('MRBM-2',9):1.00,('MRBM-2',10):0.90,
                                                      ('MRBM-2',11):0.95,('MRBM-2',12):0.96,('MRBM-2',13):0.91,('MRBM-2',14):0.98,('MRBM-2',15):0.99,
                                                      ('MRBM-2',16):0.99,('MRBM-2',17):1.00,('MRBM-2',18):1.00,('MRBM-2',19):1.00,('MRBM-2',20):1.00                                                     
                                                      });

#### Total number of weapons available per type w
model.alpha = pyomo.Param(model.w,initialize = {'ICBM':200,'MRBM-1':100,'LR-Bomber':300,'F-Bomber':150,'MRBM-2':250});

#### Military value of targets per type t
model.beta = pyomo.Param(model.t,initialize = {1:60,2:50,3:50,4:75,5:40,6:60,7:35,8:30,9:25,10:150,
                                               11:30,12:45,13:125,14:200,15:200,16:130,17:100,18:100,19:100,20:150});

#### Minimum number of weapons to be assigned to specified targets
model.gamma = pyomo.Param(model.t_spec,initialize = {1:30,6:100,10:40,14:50,15:70,16:35,20:10});

#### Constraint declaration
def rule1(model,w):
  return sum(model.x[w,t] for t in model.t) <= model.alpha[w]
model.eq1 = pyomo.Constraint(model.w,rule = rule1, doc = 'upper limit on number of weapons assigned based on availability');

def rule2(model,t_spec):
  return sum(model.x[w,t_spec] for w in model.w) >= model.gamma[t_spec]
model.eq2 = pyomo.Constraint(model.t_spec,rule = rule2, doc = 'lower limit on weapons assigned');

#### Objective function
#### Product function
from functools import reduce
import operator
def prod(iterable):
  return reduce(operator.mul,iterable,1)

model.obj = pyomo.Objective(expr = sum(model.beta[t]*(1-prod(model.phi[w,t]**model.x[w,t] for w in model.w)) 
                            for t in model.t),sense = pyomo.maximize)

# Solve statement
results = pyomo.SolverFactory('ipopt',executable = '/content/ipopt').solve(model);

results.write()
print("\n RESULTS \n");
print("Total damage = ",model.obj(),"\n");
for w in model.w:
  for t in model.t:
    print("Number of weapons of type ",w," assigned to target ",t,"=>",model.x[w,t]()," \n");

# ==========================================================
# = Solver Results                                         =
# ==========================================================
# ----------------------------------------------------------
#   Problem Information
# ----------------------------------------------------------
Problem: 
- Lower bound: -inf
  Upper bound: inf
  Number of objectives: 1
  Number of constraints: 12
  Number of variables: 100
  Sense: unknown
# ----------------------------------------------------------
#   Solver Information
# ----------------------------------------------------------
Solver: 
- Status: ok
  Message: Ipopt 3.12.13\x3a Optimal Solution Found
  Termination condition: optimal
  Id: 0
  Error rc: 0
  Time: 0.04041028022766113
# ----------------------------------------------------------
#   Solution Information
# ----------------------------------------------------------
Solution: 
- number of solutions: 0
  number of solutions displayed: 0

 RE